In [9]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [10]:
from datetime import date, datetime
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions as F

In [11]:
cdf = spark.read.csv('/dataframe/a_class_info.csv', header=True)
cdf.printSchema()
cdf.show(3)

root
 |-- class_cd: string (nullable = true)
 |-- school: string (nullable = true)
 |-- class_std_cnt: string (nullable = true)
 |-- loc: string (nullable = true)
 |-- school_type: string (nullable = true)
 |-- teaching_type: string (nullable = true)

+--------+------+-------------+--------+-----------+-------------+
|class_cd|school|class_std_cnt|     loc|school_type|teaching_type|
+--------+------+-------------+--------+-----------+-------------+
|     6OL| ANKYI|           20|   Urban| Non-public|     Standard|
|     ZNS| ANKYI|           21|   Urban| Non-public|     Standard|
|     2B1| CCAAW|           18|Suburban| Non-public| Experimental|
+--------+------+-------------+--------+-----------+-------------+
only showing top 3 rows



In [12]:
cdf.columns

['class_cd', 'school', 'class_std_cnt', 'loc', 'school_type', 'teaching_type']

### 각 반 학생수(class_std_cnt)가 평균 반 학생수보다 많은 class의 data를 추출하시오

['class_cd', 'school', 'class_std_cnt', 'loc', 'school_type', 'teaching_type']
반코드        학교이름   반 학생수       지역    학교형태       교육형태

In [13]:
# view 생성
cdf.createOrReplaceTempView('class')

In [19]:
spark.sql('''
    select * from class
    where class_std_cnt >= (select avg(class_std_cnt)
                            from class)
''').show(5)

+--------+------+-------------+--------+-----------+-------------+
|class_cd|school|class_std_cnt|     loc|school_type|teaching_type|
+--------+------+-------------+--------+-----------+-------------+
|     1Q1| CUQAM|           28|   Urban|     Public|     Standard|
|     BFY| CUQAM|           27|   Urban|     Public|     Standard|
|     OMI| CUQAM|           28|   Urban|     Public|     Standard|
|     X6Z| CUQAM|           24|   Urban|     Public| Experimental|
|     2AP| DNQDD|           27|Suburban|     Public|     Standard|
+--------+------+-------------+--------+-----------+-------------+
only showing top 5 rows



In [17]:
# where 절에서 평균 반 학생수를 구하고 그 값보다 많은 반을 추출
# 에러발생 : where절에서는 집계함수 사용 불가
# spark.sql('''
#     select * from class
#     where class_std_cnt >= avg(class_std_cnt) and class_std_cnt is not null
# ''')

- subquery 사용
  1. subquery 이용 평균 반 학생수를 전달하기
  2. where 절에서 반 학생수가 1에서 전달받은 값보다 큰 data 추출

In [20]:
# 1. subquery - 평균 반 학생수 select 하는 쿼리(집계함수는 select 절에만 사용가능)
spark.sql('''select avg(class_std_cnt) 
            from class 
            where class_std_cnt is not null
''').show()

+------------------+
|avg(class_std_cnt)|
+------------------+
|21.828282828282827|
+------------------+



In [21]:
# 2. 1의 query를 where에 비교 대상으로 사용(subquery 적용)
spark.sql('''
    select * 
    from class
    where (class_std_cnt >= (select avg(class_std_cnt) 
                            from class))
                            and (class_std_cnt is not null)
''').show()

+--------+------+-------------+--------+-----------+-------------+
|class_cd|school|class_std_cnt|     loc|school_type|teaching_type|
+--------+------+-------------+--------+-----------+-------------+
|     1Q1| CUQAM|           28|   Urban|     Public|     Standard|
|     BFY| CUQAM|           27|   Urban|     Public|     Standard|
|     OMI| CUQAM|           28|   Urban|     Public|     Standard|
|     X6Z| CUQAM|           24|   Urban|     Public| Experimental|
|     2AP| DNQDD|           27|Suburban|     Public|     Standard|
|     ROP| DNQDD|           28|Suburban|     Public| Experimental|
|     XXJ| DNQDD|           27|Suburban|     Public|     Standard|
|     HCB| GJJHK|           22|Suburban|     Public|     Standard|
|     NOR| GJJHK|           27|Suburban|     Public| Experimental|
|     ZDT| GJJHK|           27|Suburban|     Public|     Standard|
|     ENO| GOKXL|           22|   Rural|     Public| Experimental|
|     TSA| GOKXL|           23|   Rural|     Public| Experimen

### df method을 사용해서 subquery와 같은 효과

In [27]:
# 1. 평균값 구하기
# 1-2. 연산식이 아니라 평균값을 수치로 받아와야함
cdf.select(avg('class_std_cnt'))
cdf.select(avg('class_std_cnt')).show()

cdf.select(avg('class_std_cnt')).collect()[0][0]

DataFrame[avg(class_std_cnt): double]

+------------------+
|avg(class_std_cnt)|
+------------------+
|21.828282828282827|
+------------------+



21.828282828282827

In [34]:
cdf.select(avg('class_std_cnt')).collect()
type(cdf.select(avg('class_std_cnt')).collect())
cdf.select(avg('class_std_cnt')).collect()[0]
type(cdf.select(avg('class_std_cnt')).collect()[0])
cdf.select(avg('class_std_cnt')).collect()[0][0]
type(cdf.select(avg('class_std_cnt')).collect()[0][0])

[Row(avg(class_std_cnt)=21.828282828282827)]

list

Row(avg(class_std_cnt)=21.828282828282827)

pyspark.sql.types.Row

21.828282828282827

float

- spark.sql.dataframe.DataFrame 객체도 RDD기반임
    - collect() 사용 가능
        - 객체 타입에 따라 index 사용이 가능 : 실제값을 추출할 수 있음
    - show() 는 출력만 진행함

In [44]:
# 2. 1에서 구한 평균값을 활용해서 평균보다 많은 학생수 추출
# 아래 코드의 cdf.select(avg("class_std_cnt") 연산식은 반환값이 수치가 아니고 객체 // 객체 id가 결과로 나옴
# where절을 진행할 수 없음 - 에러
# cdf.select("*") \
#     .where(cdf.class_std_cnt >= cdf.select(avg("class_std_cnt"))) \
#     .show()

cdf.select('*') \
    .where(cdf.class_std_cnt >= cdf.select(avg('class_std_cnt')).collect()[0][0]) \
    .show(2)

복잡한 연산도 해보자

#### case 1. 소속된 반의 개수가 3개 이상인 학교들 중 학생 숫자가 가장 적은 학교의 학생 수를 구해보자
#### 단, 학교가 null인 데이터는 제외한다

In [44]:
# from절 subquery : inline view

cdf.createOrReplaceTempView('class')

spark.sql('''
    select min(std_cnt) 
    from (select school, sum(class_std_cnt) as std_cnt 
            from class 
            where school is not null
            group by school
            having count(school) >= 3 )
''').show(3)

+------------+
|min(std_cnt)|
+------------+
|        46.0|
+------------+



+------------+
|min(std_cnt)|
+------------+
|        46.0|
+------------+



#### case 1_1. 소속된 반의 개수가 3개 이상인 학교들 중 학생 숫자가 가장 적은 학교의 학생 수와 학교명을 추출하자
#### 단, 학교가 null인 데이터는 제외한다

### sql query

In [74]:
spark.sql(''' select school, std_tot
            from (select school, sum(class_std_cnt) as std_tot from class group by school having school is not null and count(school)>=3)
            where std_tot == (select min(std_tot) as min_stdtot
                              from(select school, sum(class_std_cnt) as std_tot 
                                   from class 
                                   group by school 
                                   having school is not null and count(school)>=3))  
    
    ''').show()

+------+-------+
|school|std_tot|
+------+-------+
| FBUMG|   46.0|
+------+-------+



### df moethod 사용

+------+-------+
|school|std_cnt|
+------+-------+
| VHDHF|   51.0|
| LAYPA|   57.0|
| GOOBU|  158.0|
| UUUQX|   84.0|
| CIMBB|   74.0|
| UKPGS|  128.0|
| UAGPU|   87.0|
| CCAAW|  109.0|
| FBUMG|   46.0|
| ZOWMK|  117.0|
| ZMNYA|   69.0|
| QOQTS|  149.0|
| CUQAM|  107.0|
| OJOBU|   81.0|
| GOKXL|   64.0|
| GJJHK|  118.0|
| KZKKE|  111.0|
| DNQDD|  122.0|
| VKWQH|  100.0|
| IDGFP|   94.0|
+------+-------+
only showing top 20 rows



+------+-------+
|school|std_cnt|
+------+-------+
| FBUMG|   46.0|
+------+-------+



### case2. 1.지역에 따른 학교로 분류하고 분류된 학교의 class_cd가 2개 초과인 학교별로 반의 학생수가 가장 작은 반의 학생수를 구하시오
### 위에서 구한 학생수중 가장 큰 값은 얼마인가?
1. 지역에 따른 학교로 분류하고 학교의 class_cd가 2개 초과인 학교들을 추출
2. 추출된 학교들에서 학생수가 가장 작은 반의 학생수 추출
3. 2번에서 추출된 학생수들 중 가장 큰 수 추출

### sql query 활용

In [127]:
# 참고
# Urban 지역의 학교 VVTVA는 반수가 4개 반 학생수 중 가장 작은 수는 25
cdf.select('*').where((cdf.school=='VVTVA') & (cdf.loc=='Urban')).show()

+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     7BL| VVTVA|           29|Urban|     Public|     Standard|
|     A93| VVTVA|           30|Urban|     Public| Experimental|
|     TB5| VVTVA|           25|Urban|     Public|     Standard|
|     YTB| VVTVA|           30|Urban|     Public| Experimental|
+--------+------+-------------+-----+-----------+-------------+



In [126]:
# 1. 지역에 따른 학교로 분류하고 학교의 class_cd가 2개 초과인 학교들을 추출


+--------+------+---+
|     loc|school|cnt|
+--------+------+---+
|   Rural| VHDHF|  3|
|   Urban| GOOBU|  6|
|Suburban| ZMNYA|  3|
|   Urban| ZOWMK|  4|
|    null|  null|  3|
|Suburban| UUUQX|  5|
|Suburban| DNQDD|  5|
|   Urban| CUQAM|  4|
|   Urban| IDGFP|  5|
|   Rural| GOKXL|  3|
|   Rural| KZKKE|  5|
|   Rural| VKWQH|  5|
|Suburban| CCAAW|  6|
|   Rural| LAYPA|  3|
|   Urban| QOQTS|  6|
|Suburban| UAGPU|  4|
|   Rural| FBUMG|  3|
|   Urban| VVTVA|  4|
|Suburban| UKPGS|  6|
|Suburban| GJJHK|  5|
|   Rural| OJOBU|  4|
|   Urban| CIMBB|  4|
+--------+------+---+



In [128]:
# Urban 지역의 학교 VVTVA는 반수가 4개 반 학생수 중 가장 작은 수는 25
cdf.select('*').where((cdf.school=='VVTVA') & (cdf.loc=='Urban')).show()

+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     7BL| VVTVA|           29|Urban|     Public|     Standard|
|     A93| VVTVA|           30|Urban|     Public| Experimental|
|     TB5| VVTVA|           25|Urban|     Public|     Standard|
|     YTB| VVTVA|           30|Urban|     Public| Experimental|
+--------+------+-------------+-----+-----------+-------------+



In [124]:
# 2. 추출된 학교들에서 학생수가 가장 작은 반의 학생수 추출


+--------+------+---+-------+
|     loc|school|cnt|min_std|
+--------+------+---+-------+
|   Rural| FBUMG|  3|     14|
|   Rural| GOKXL|  3|     19|
|   Rural| KZKKE|  5|     20|
|   Rural| LAYPA|  3|     17|
|   Rural| OJOBU|  4|     17|
|   Rural| VHDHF|  3|     15|
|   Rural| VKWQH|  5|     18|
|Suburban| CCAAW|  6|     15|
|Suburban| DNQDD|  5|     20|
|Suburban| GJJHK|  5|     21|
|Suburban| UAGPU|  4|     21|
|Suburban| UKPGS|  6|     18|
|Suburban| UUUQX|  5|     15|
|Suburban| ZMNYA|  3|     22|
|   Urban| CIMBB|  4|     17|
|   Urban| CUQAM|  4|     24|
|   Urban| GOOBU|  6|     24|
|   Urban| IDGFP|  5|     17|
|   Urban| QOQTS|  6|     22|
|   Urban| VVTVA|  4|     25|
|   Urban| ZOWMK|  4|     27|
+--------+------+---+-------+



In [125]:
# 3. 2번에서 추출된 학생수들 중 가장 큰 수 추출

# 즉, Urban지역의 ZOWMK 학교는 한반의 학생수 27명이 가장 작은데, 다른 지역의 학교들의 학생수가 가장 작은 반은 27명보다 작다

+------------+
|max(min_std)|
+------------+
|          27|
+------------+



### df의 메서드 활용

In [80]:
# Urban 지역의 학교 VVTVA는 반수가 4개 반 학생수 중 가장 작은 수는 25
cdf.select('*').where((cdf.school=='VVTVA') & (cdf.loc=='Urban')).show()

+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     7BL| VVTVA|           29|Urban|     Public|     Standard|
|     A93| VVTVA|           30|Urban|     Public| Experimental|
|     TB5| VVTVA|           25|Urban|     Public|     Standard|
|     YTB| VVTVA|           30|Urban|     Public| Experimental|
+--------+------+-------------+-----+-----------+-------------+



In [91]:
# 1. 지역에 따른 학교로 분류하고 학교의 class_cd가 2개 초과인 학교들을 추출

        

+--------+------+------+
|     loc|school|cnt_cd|
+--------+------+------+
|   Rural| VHDHF|     3|
|   Urban| GOOBU|     6|
|Suburban| ZMNYA|     3|
|   Urban| ZOWMK|     4|
|Suburban| UUUQX|     5|
|Suburban| DNQDD|     5|
|   Urban| CUQAM|     4|
|   Urban| IDGFP|     5|
|   Rural| GOKXL|     3|
|   Rural| KZKKE|     5|
|   Rural| VKWQH|     5|
|Suburban| CCAAW|     6|
|   Rural| LAYPA|     3|
|   Urban| QOQTS|     6|
|Suburban| UAGPU|     4|
|   Rural| FBUMG|     3|
|   Urban| VVTVA|     4|
|Suburban| UKPGS|     6|
|Suburban| GJJHK|     5|
|   Rural| OJOBU|     4|
|   Urban| CIMBB|     4|
+--------+------+------+



In [108]:
# Urban 지역의 학교 VVTVA는 반수가 4개 반 학생수 중 가장 작은 수는 25
cdf.select('*').where((cdf.school=='VVTVA') & (cdf.loc=='Urban')).show()

+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     7BL| VVTVA|           29|Urban|     Public|     Standard|
|     A93| VVTVA|           30|Urban|     Public| Experimental|
|     TB5| VVTVA|           25|Urban|     Public|     Standard|
|     YTB| VVTVA|           30|Urban|     Public| Experimental|
+--------+------+-------------+-----+-----------+-------------+



In [92]:
# 2. 추출된 학교들에서 학생수가 가장 작은 반의 학생수 추출
# 지역에따른 학교의 반들에 대해 가장 학생수가 작은반의 학생수


+--------+------+------+-----------+
|     loc|school|cnt_cd|min_std_cnt|
+--------+------+------+-----------+
|   Rural| FBUMG|     3|         14|
|   Rural| GOKXL|     3|         19|
|   Rural| KZKKE|     5|         20|
|   Rural| LAYPA|     3|         17|
|   Rural| OJOBU|     4|         17|
|   Rural| VHDHF|     3|         15|
|   Rural| VKWQH|     5|         18|
|Suburban| CCAAW|     6|         15|
|Suburban| DNQDD|     5|         20|
|Suburban| GJJHK|     5|         21|
|Suburban| UAGPU|     4|         21|
|Suburban| UKPGS|     6|         18|
|Suburban| UUUQX|     5|         15|
|Suburban| ZMNYA|     3|         22|
|   Urban| CIMBB|     4|         17|
|   Urban| CUQAM|     4|         24|
|   Urban| GOOBU|     6|         24|
|   Urban| IDGFP|     5|         17|
|   Urban| QOQTS|     6|         22|
|   Urban| VVTVA|     4|         25|
|   Urban| ZOWMK|     4|         27|
+--------+------+------+-----------+



In [107]:
# 3. 2번에서 추출된 학생수들 중 가장 많은 수 추출
# # 지역에따른 학교의 반들에 대해 가장 학생수가 작은반의 학생수에서 가장 큰 수 
# 즉, Urban지역의 ZOWMK 학교는 한반의 학생수 27명이 가장 작은데, 다른 지역의 학교들의 학생수가 가장 작은 반은 27명보다 작다


+----------------+
|max(min_std_cnt)|
+----------------+
|              27|
+----------------+



### case3. 지역에 따른 학교로 분류하고 분류된 학교의 class_cd가 2개 초과인 학교에서 학교별 가장 작은 학생수들을 추출 그 중에서 가장 큰 수를 구하시오(27)
### 구한 수보다 학생수가 더 많은 반과 학생수를 cdf 전체 데이터에서 추출하시오

#### sql 쿼리

In [130]:

spark.sql('select count(*) from class').show()
# Urban지역의 ZOWMK 학교의 학생수가 가장 작은반보다 학생수가 많은 반은  전체 데이터 102개의 class 중 13 클래스이다

+--------+-------------+
|class_cd|class_std_cnt|
+--------+-------------+
|     1Q1|           28|
|     OMI|           28|
|     ROP|           28|
|     18K|           31|
|     HKF|           28|
|     0N7|           28|
|     SUR|           28|
|     7BL|           29|
|     A93|           30|
|     YTB|           30|
|     Q0E|           30|
|     QA2|           30|
|     ZBH|           30|
+--------+-------------+

+--------+
|count(1)|
+--------+
|     102|
+--------+



#### df 메서드

In [110]:

cdf.count()
# Urban지역의 ZOWMK 학교의 학생수가 가장 작은반보다 학생수가 많은 반은 전체 데이터 102개의 class 중 13 클래스이다

+--------+-------------+
|class_cd|class_std_cnt|
+--------+-------------+
|     1Q1|           28|
|     OMI|           28|
|     ROP|           28|
|     18K|           31|
|     HKF|           28|
|     0N7|           28|
|     SUR|           28|
|     7BL|           29|
|     A93|           30|
|     YTB|           30|
|     Q0E|           30|
|     QA2|           30|
|     ZBH|           30|
+--------+-------------+



102

#### case 4. 시골지역의 사립학교중 표준교육을 진행하는 학교들의 평균 학생수보다 학생수가 더 많은 도시 지역의 공립학교면서 특수교육을 진행하는 학교를 추출하시오

In [ ]:
# 시골지역의 사립학교중 표준교육을 진행하는 학교들의 평균 학생수


+------------------+
|avg(class_std_cnt)|
+------------------+
|20.928571428571427|
+------------------+



+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     98D| IDGFP|           21|Urban| Non-public| Experimental|
|     1VD| KFZMY|           27|Urban| Non-public| Experimental|
+--------+------+-------------+-----+-----------+-------------+



In [138]:
# 시골지역의 사립학교중 표준교육을 진행하는 학교들의 평균 학생수

+---+
|avg|
+---+
| 21|
+---+



21

+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     98D| IDGFP|           21|Urban| Non-public| Experimental|
|     1VD| KFZMY|           27|Urban| Non-public| Experimental|
+--------+------+-------------+-----+-----------+-------------+

